In [1]:
print("EHorizon")

EHorizon


In [2]:
DATA_PATH = "../data/raw/deu-eng/deu.txt"

Data is collected from manything.org

In [3]:
#reading .txt file
with open(DATA_PATH, encoding='utf-8') as f:
    lines = f.readlines()

# View the first 5 lines
for line in lines[:5]:
    print(line.strip())

Go.	Geh.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8597805 (Roujin)
Hi.	Hallo!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #380701 (cburgmer)
Hi.	Grüß Gott!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #659813 (Esperantostern)
Run!	Lauf!	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #941078 (Fingerhut)
Run.	Lauf!	CC-BY 2.0 (France) Attribution: tatoeba.org #4008918 (JSakuragi) & #941078 (Fingerhut)


In [4]:
#removing metadata tags from data
cleaned_pairs = []
with open(DATA_PATH, encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) >= 2:  # Ensure at least English and German
            english = parts[0].strip()
            german = parts[1].strip()
            cleaned_pairs.append([english, german])

In [5]:
cleaned_pairs[:5]

[['Go.', 'Geh.'],
 ['Hi.', 'Hallo!'],
 ['Hi.', 'Grüß Gott!'],
 ['Run!', 'Lauf!'],
 ['Run.', 'Lauf!']]

In [25]:
len(cleaned_pairs)

277891

In [26]:
#converting to a dataframe
import pandas as pd

df = pd.DataFrame(cleaned_pairs, columns=['English', 'German'])

df.head()

,English,German
0,Go.,Geh.
1,Hi.,Hallo!
2,Hi.,Grüß Gott!
3,Run!,Lauf!
4,Run.,Lauf!


In [27]:
#basic cleaning
null_values = df.isnull().sum()
duplicate_values = df.duplicated().sum()

print("Null Values:")
print(null_values)
print("\nDuplicate Values:")
print(duplicate_values)

Null Values:
English    0
German     0
dtype: int64

Duplicate Values:
0


In [28]:
!pip install contractions

You should consider upgrading via the 'D:\AI Projects\LangConv\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [29]:
#imports
import re
import unicodedata
import string
import contractions

In [30]:
#example of raw text cleaning
sample_text = "Hello, how are you? You're amazing! Don't go!!! "
sample_text = contractions.fix(sample_text)
print("Contractions example : ",sample_text)
sample_text = sample_text.lower()
print("Lowercase example : ", sample_text)
exclude = ''.join(ch for ch in string.punctuation if ch not in ['?', '.', '!'])
sample_text = re.sub(rf"[{re.escape(exclude)}]", "", sample_text)
print("Punctuation example : ",sample_text)
sample_text =  re.sub(r"\s+", " ", sample_text).strip()
print("Whitespace example : ", sample_text)

Contractions example :  Hello, how are you? You are amazing! Do not go!!! 
Lowercase example :  hello, how are you? you are amazing! do not go!!! 
Punctuation example :  hello how are you? you are amazing! do not go!!! 
Whitespace example :  hello how are you? you are amazing! do not go!!!


In [31]:
def clean_text(text, lowercase=True, remove_punct=True, expand_contr=True):
    text = unicodedata.normalize('NFKC', text)

    if expand_contr:
        text = contractions.fix(text)

    if lowercase:
        text = text.lower()

    if remove_punct:
        exclude = ''.join(ch for ch in string.punctuation if ch not in ['?', '.', '!'])
        text = re.sub(rf"[{re.escape(exclude)}]", "", text)

    text = re.sub(r"\s+", " ", text).strip()

    return text


In [32]:
# Apply preprocessing to English column
df['English_clean'] = df['English'].apply(
    lambda x: clean_text(x, lowercase=True, remove_punct=True, expand_contr=True)
)

# Apply preprocessing to German column (no contractions expansion for German)
df['German_clean'] = df['German'].apply(
    lambda x: clean_text(x, lowercase=True, remove_punct=True, expand_contr=False)
)


In [33]:
#dropping original columns
df.drop(columns=['English', 'German'], inplace=True)

In [34]:
df.sample(5)

,English_clean,German_clean
91534,i am very happy in boston.,ich bin sehr glücklich in boston.
257412,you should not laugh at tom when he makes mist...,du darfst nicht über tom lachen wenn er einen ...
125732,there is not anyone in there.,da ist niemand.
74246,tom put his glasses on.,tom setzte sich die brille auf.
49191,his mother was right.,seine mutter hatte recht.


In [35]:
# Save source (e.g., English)
df['English_clean'].to_csv('../data/processed/src_train.txt', index=False, header=False)

# Save target (e.g., German)
df['German_clean'].to_csv('../data/processed/trg_train.txt', index=False, header=False)

Tokenization

I am using SentencePiece Tokenizer using  Byte Pair Encoding (BPE) algorithm

In [36]:
!pip install sentencepiece

You should consider upgrading via the 'D:\AI Projects\LangConv\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [37]:
import sentencepiece as spm

In [38]:
# English tokenizer
spm.SentencePieceTrainer.train(
    input='../data/processed/src_train.txt',
    model_prefix='../data/processed/spm_en',
    vocab_size=16000,
    model_type='bpe',
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3,
    user_defined_symbols=[".", "!", "?"]
)

# German tokenizer
spm.SentencePieceTrainer.train(
    input='../data/processed/trg_train.txt',
    model_prefix='../data/processed/spm_de',
    vocab_size=16000,
    model_type='bpe',
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3,
    user_defined_symbols=[".", "!", "?"]
)

In [39]:
# Loading trained Tokenization models

sp_en = spm.SentencePieceProcessor()
sp_en.load('../data/processed/spm_en.model')

sp_de = spm.SentencePieceProcessor()
sp_de.load('../data/processed/spm_de.model')

True

In [40]:
sample_text_en = "Hello, how are you? You're amazing! Don't go!!!"
sample_text_de = "Hallo, wie geht es dir? Du bist erstaunlich! Geh nicht!!!"

# Tokenizing sample text
tokens_en = sp_en.encode(sample_text_en, out_type=str)
tokens_de = sp_de.encode(sample_text_de, out_type=str)

print("English Tokens:", tokens_en)
print("German Tokens:", tokens_de)

# Detokenizing sample text
# detokenized_en = sp_en.decode(tokens_en)
# detokenized_de = sp_de.decode(tokens_de)
# print("Detokenized English:", detokenized_en)
# print("Detokenized German:", detokenized_de)

English Tokens: ['▁', 'H', 'ell', 'o', ',', '▁how', '▁are', '▁you', '?', '▁', 'Y', 'ou', "'", 're', '▁amazing', '!', '▁', 'D', 'on', "'", 't', '▁go', '!', '!', '!']
German Tokens: ['▁', 'H', 'all', 'o', ',', '▁wie', '▁geht', '▁es', '▁dir', '?', '▁', 'D', 'u', '▁bist', '▁erstaunlich', '!', '▁', 'G', 'eh', '▁nicht', '!', '!', '!']


In [41]:
# Updating the DataFrame with tokenized columns
df['src_tokens'] = df['English_clean'].apply(lambda x: sp_en.encode(x, out_type=str))
df['trg_tokens'] = df['German_clean'].apply(lambda x: sp_de.encode(x, out_type=str))

In [42]:
df.sample(5)

,English_clean,German_clean,src_tokens,trg_tokens
197206,she accused him of stealing her car.,sie beschuldigte ihn ihr auto gestohlen zu haben.,"[▁she, ▁accused, ▁him, ▁of, ▁stealing, ▁her, ▁...","[▁sie, ▁beschuldigte, ▁ihn, ▁ihr, ▁auto, ▁gest..."
95218,tom likes playing soccer.,tom spielt gerne fußball.,"[▁tom, ▁likes, ▁playing, ▁soccer, .]","[▁tom, ▁spielt, ▁gerne, ▁fußball, .]"
24596,go get some water.,geh etwas wasser holen.,"[▁go, ▁get, ▁some, ▁water, .]","[▁geh, ▁etwas, ▁wasser, ▁holen, .]"
206833,where have you been? i have been worried.,wo sind sie gewesen? ich habe mir sorgen gemacht.,"[▁where, ▁have, ▁you, ▁been, ?, ▁i, ▁have, ▁be...","[▁wo, ▁sind, ▁sie, ▁gewesen, ?, ▁ich, ▁habe, ▁..."
126823,tom is an aggressive driver.,tom ist ein aggressiver autofahrer.,"[▁tom, ▁is, ▁an, ▁aggressive, ▁driver, .]","[▁tom, ▁ist, ▁ein, ▁aggress, iver, ▁autofahrer..."


In [43]:
#Calculating the average length of tokens in the source and target languages
avg_src_length = df['src_tokens'].apply(len).mean()
max_src_length = df['src_tokens'].apply(len).max()
min_src_length = df['src_tokens'].apply(len).min()

avg_trg_length = df['trg_tokens'].apply(len).mean()
max_trg_length = df['trg_tokens'].apply(len).max()
min_trg_length = df['trg_tokens'].apply(len).min()

print(f"Max Source Tokens Length: {max_src_length}")
print(f"Min Source Tokens Length: {min_src_length}")
print("-" * 50)
print(f"Max Target Tokens Length: {max_trg_length}")
print(f"Min Target Tokens Length: {min_trg_length}")
print("-" * 50)
print(f"Average Source Tokens Length: {avg_src_length}")
print(f"Average Target Tokens Length: {avg_trg_length}")

Max Source Tokens Length: 105
Min Source Tokens Length: 2
--------------------------------------------------
Max Target Tokens Length: 100
Min Target Tokens Length: 2
--------------------------------------------------
Average Source Tokens Length: 7.6800724024887455
Average Target Tokens Length: 7.684754094231192


In [47]:
# Converting tokens to IDs
df['src_ids'] = df['English_clean'].apply(lambda s: sp_en.encode(s, out_type=int))
df['trg_ids'] = df['German_clean'].apply(lambda s: sp_de.encode(s, out_type=int))



In [48]:
df.sample(5)

,English_clean,German_clean,src_tokens,trg_tokens,src_ids,trg_ids
152778,i know that tom is a smart boy.,ich weiß dass tom ein kluger junge ist.,"[▁i, ▁know, ▁that, ▁tom, ▁is, ▁a, ▁smart, ▁boy...","[▁ich, ▁weiß, ▁dass, ▁tom, ▁ein, ▁kluger, ▁jun...","[8, 121, 56, 22, 32, 9, 1850, 798, 4]","[27, 212, 88, 28, 41, 8481, 1164, 49, 4]"
232253,tom crawled into bed just before midnight.,tom ist erst kurz vor mitternacht ins bett gek...,"[▁tom, ▁crawled, ▁into, ▁bed, ▁just, ▁before, ...","[▁tom, ▁ist, ▁erst, ▁kurz, ▁vor, ▁mitternacht,...","[22, 7405, 471, 674, 263, 465, 3032, 4]","[28, 49, 939, 1283, 177, 4021, 567, 919, 564, ..."
120492,he asked for a lot of money.,er forderte viel geld.,"[▁he, ▁asked, ▁for, ▁a, ▁lot, ▁of, ▁money, .]","[▁er, ▁forderte, ▁viel, ▁geld, .]","[46, 467, 88, 9, 343, 69, 364, 4]","[57, 6833, 193, 394, 4]"
262632,he sent me a letter asking if the book had rea...,er schickte mir einen brief in dem er mich fra...,"[▁he, ▁sent, ▁me, ▁a, ▁letter, ▁asking, ▁if, ▁...","[▁er, ▁schickte, ▁mir, ▁einen, ▁brief, ▁in, ▁d...","[46, 1082, 78, 9, 776, 1733, 220, 23, 321, 209...","[57, 4086, 114, 171, 916, 77, 181, 57, 127, 11..."
175717,tom knows where to draw the line.,tom weiß wo er einen schlußstrich ziehen muß.,"[▁tom, ▁knows, ▁where, ▁to, ▁draw, ▁the, ▁line...","[▁tom, ▁weiß, ▁wo, ▁er, ▁einen, ▁schl, uß, str...","[22, 552, 264, 10, 2021, 23, 2061, 4]","[28, 212, 124, 57, 171, 255, 5768, 15234, 2148..."


In [49]:
new_df = df.copy()

#drop original columns
new_df.drop(columns=['English_clean', 'German_clean'], inplace=True)

In [50]:
bos_id_en = sp_en.bos_id()   # English BOS, usually 2
eos_id_en = sp_en.eos_id()

bos_id_de = sp_de.bos_id()   # German BOS, usually 2
eos_id_de = sp_de.eos_id()

print("BOS ID for English:", bos_id_en)
print("EOS ID for English:", eos_id_en)
print("BOS ID for German:", bos_id_de)
print("EOS ID for German:", eos_id_de)

BOS ID for English: 2
EOS ID for English: 3
BOS ID for German: 2
EOS ID for German: 3


In [51]:
# adding BOS and EOS tokens to the source and target IDs
new_df['src_ids'] = new_df['src_ids'].apply(lambda x: [bos_id_en] + x + [eos_id_en])
new_df['trg_ids'] = new_df['trg_ids'].apply(lambda x: [bos_id_de] + x + [eos_id_de])

In [52]:
new_df.sample(5)

,src_tokens,trg_tokens,src_ids,trg_ids
188826,"[▁i, ▁will, ▁have, ▁a, ▁coffee, ▁and, ▁a, ▁cro...","[▁ich, ▁nehme, ▁einen, ▁kaffee, ▁und, ▁ein, ▁h...","[2, 8, 102, 65, 9, 746, 130, 9, 15017, 4, 3]","[2, 27, 1866, 171, 867, 140, 41, 416, 3535, 4, 3]"
121561,"[▁i, ▁do, ▁not, ▁know, ▁how, ▁they, ▁do, ▁it, .]","[▁ich, ▁weiß, ▁nicht, ▁wie, ▁sie, ▁das, ▁mache...","[2, 8, 35, 37, 121, 153, 203, 35, 63, 4, 3]","[2, 27, 212, 44, 108, 54, 38, 315, 4, 3]"
212978,"[▁what, ▁time, ▁do, ▁you, ▁get, ▁up, ▁on, ▁sch...","[▁um, ▁wieviel, ▁uhr, ▁stehst, ▁du, ▁auf, ▁wen...","[2, 94, 194, 35, 21, 179, 196, 81, 11470, 59, ...","[2, 176, 2402, 626, 5440, 51, 94, 237, 581, 49..."
52914,"[▁that, ▁is, ▁pretty, ▁normal, .]","[▁das, ▁ist, ▁ganz, ▁normal, .]","[2, 56, 32, 872, 2553, 4, 3]","[2, 38, 49, 356, 1647, 4, 3]"
186386,"[▁chocolate, ▁ice, ▁cream, ▁is, ▁my, ▁favorite...","[▁schokoladeneis, ▁esse, ▁ich, ▁am, ▁allerlieb...","[2, 2268, 1527, 2125, 32, 99, 1017, 4, 3]","[2, 10897, 1865, 27, 297, 10882, 4, 3]"


In [53]:
new_df['src_len'] = new_df['src_ids'].apply(len)
new_df['trg_len'] = new_df['trg_ids'].apply(len)


In [54]:
new_df.sample(5)

,src_tokens,trg_tokens,src_ids,trg_ids,src_len,trg_len
125301,"[▁that, ▁is, ▁why, ▁i, ▁do, ▁not, ▁like, ▁tom, .]","[▁deshalb, ▁mag, ▁ich, ▁tom, ▁nicht, .]","[2, 56, 32, 193, 8, 35, 37, 133, 22, 4, 3]","[2, 3252, 403, 27, 28, 44, 4, 3]",11,8
163846,"[▁let, ▁us, ▁start, ▁as, ▁soon, ▁as, ▁he, ▁com...","[▁lasst, ▁uns, ▁beginnen, ▁sobald, ▁er, ▁kommt...","[2, 254, 149, 551, 127, 570, 127, 46, 1382, 4, 3]","[2, 1410, 166, 3527, 2530, 57, 569, 4, 3]",11,9
127910,"[▁tom, ▁wondered, ▁the, ▁same, ▁thing, .]","[▁tom, ▁fragte, ▁sich, ▁dasselbe, .]","[2, 22, 2455, 23, 742, 409, 4, 3]","[2, 28, 1137, 113, 4580, 4, 3]",8,7
214296,"[▁he, ▁does, ▁not, ▁know, ▁who, ▁built, ▁those...","[▁er, ▁weiß, ▁nicht, ▁wer, ▁die, ▁häuser, ▁dor...","[2, 46, 176, 37, 121, 213, 2124, 863, 3166, 4, 3]","[2, 57, 212, 44, 139, 52, 4221, 548, 3464, 73,...",11,12
253252,"[▁do, ▁you, ▁know, ▁how, ▁much, ▁strawberries,...","[▁weißt, ▁du, ▁was, ▁die, ▁erdbeeren, ▁jetzt, ...","[2, 35, 21, 121, 153, 303, 3915, 1187, 395, 26...","[2, 818, 51, 106, 52, 5384, 338, 2518, 6, 3]",12,10


Saving the cleaned and preprocessed dataset

In [55]:
new_df.to_csv("../data/processed/cleaned_dataset.csv", index=False)